# Milestone 2

## Imports

# Quotebank preprocessing

Most of the code used for the quotebank preprocessing can be found in the classes QuotebankData and Keyword, in addition to in files for helper functions. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import random
import os
import bz2
from src.QuoteBankData import QuoteBankData
from src.Deaths import Deaths
import src.utilities.quotebank_preprocessing_utils as utils
from src.CONSTS import DATA_PATH, COLORS, KEYWORDS_JSON_FILE_PATH, KEYWORDS_FILE_PATH, BEGIN_YEAR, END_YEAR
from tqdm import tqdm
from src.utilities.synonym_utils import add_new_synonyms
from src.utilities.add_features import expand_line
from src.utilities.utils import *

[nltk_data] Downloading package wordnet to C:\Users\Henrik Øberg
[nltk_data]     Myhre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Initialize quotebank
- read keyword and synonyms for death causes
- create folders and file names for quotes that matches keywords or synonyms
- match keywords and synonyms with quotes

load database of speakers


In [2]:
quotebank = QuoteBankData("Asymmetry of News", [])
quotebank.read_keywords_from_file()
utils.create_directories_for_every_year()
quotebank.create_json_dumps_filenames_for_each_keyword()

In [3]:
#quotebank.print_pretty_keywords_filenames()

In [4]:
import warnings
import collections

warnings.filterwarnings("ignore")

"""
def save_meta_data_quotes():
    quotebank.quotes_occurrences_df = pd.DataFrame()
    for year in range(BEGIN_YEAR, END_YEAR + 1):
        dict_df_row = {"Year": year}
        for key in quotebank.keywords:
            file = key.output_filenames[year-BEGIN_YEAR]
            print(file)
            with open(file, 'rU') as f:
                quotes = pd.read_json(file, lines=True)
            # Save total number of quotes per year per topic in df
            #num_of_quotes = quotes["n_appearences"].sum()
            #occurences.append(num_of_quotes)
            #dict_df_row[key.name] = num_of_quotes
            print(quotes)
            for line_quotes in quotes:
                # Safe number of quotes per topic per year per country of url
                print(line_quotes)
                country_urls = line_quotes['country_urls']
                for country in country_urls:
                    if country in key.country_url_occurences:
                        key.country_url_occurences[country] += 1
                    else:
                        key.country_url_occurences[country] = 1
                
                # Safe number of quotes per topic per year per country of speaker
                country_speaker = line_quotes['country_speaker']
                if isinstance(country_speaker, list):        # if multiple countries, take most occuring
                    country_speaker = collections.Counter(country_speaker).most_common(1)[0][0]
                    
                if country_speaker in key.country_speaker_url_occurences:
                    key.country_speaker_url_occurences[country_speaker] +=1
                else:
                    key.country_speaker_url_occurences[country_speaker] = 1 
            
            
        quotebank.quotes_occurrences_df = quotebank.quotes_occurrences_df.append(dict_df_row, ignore_index = True)
        cols = quotebank.quotes_occurrences_df.columns
        quotebank.quotes_occurrences_df[cols] = quotebank.quotes_occurrences_df[cols].astype(int)
save_meta_data_quotes()
"""
def save_number_of_quotes():
    quotebank.quotes_occurrences_df = pd.DataFrame()
    for year in range(BEGIN_YEAR, END_YEAR + 1):
        dict_df_row = {"Year": year}
        for key in quotebank.keywords:
            file = key.output_filenames[year-BEGIN_YEAR]
            with open(file, 'rU') as f:
                quotes = pd.read_json(file, lines=True)
            print(quotes[country_speaker'])
            if len(quotes) > 0:
                num_of_quotes = quotes["n_appearances"].sum()
            else: num_of_quotes = 0
            dict_df_row[key.name] = num_of_quotes
        quotebank.quotes_occurrences_df = quotebank.quotes_occurrences_df.append(dict_df_row, ignore_index = True)
        cols = quotebank.quotes_occurrences_df.columns
        quotebank.quotes_occurrences_df[cols] = quotebank.quotes_occurrences_df[cols].astype(int)

In [5]:
save_number_of_quotes()

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
             quoteID                                          quotation  \
0  2008-09-29-004287  as the number of older americans continues to ...   

         speaker         qids                date  numOccurrences  \
0  mary c. white  [Q43737912] 2008-09-29 19:06:53               3   

                                      probas  \
0  [[mary c. white, 0.7179], [None, 0.2821]]   

                                                urls phase  country_speaker  \
0  [http://reuters.com/article/healthNews/idUSTRE...     A              NaN   

   n_appearances                                       country_urls  
0   

             quoteID                                          quotation  \
0  2009-10-11-041550  we had that one tough inning in the first game...   
1  2009-12-15-088719  we thought it was highly, highly unlikely that...   

        speaker                  qids                date  numOccurrences  \
0     lee brand           [Q28514354] 2009-10-11 22:32:20               1   
1  anne edwards  [Q4768302, Q4768304] 2009-12-15 01:50:11               1   

                                     probas  \
0       [[lee brand, 0.595], [None, 0.405]]   
1  [[anne edwards, 0.6809], [None, 0.3191]]   

                                                urls phase  \
0  [http://columbian.com/article/20091011/SPORTS0...     A   
1  [http://gleninnesexaminer.com.au/news/local/ne...     A   

                      country_speaker  n_appearances country_urls  
0                                None              1       [None]  
1  [Canada, United States of America]              1  [Australia]  
         

             quoteID                                          quotation  \
0  2010-03-23-010311  but the cystic fibrosis trust is funding trial...   

         speaker         qids                date  numOccurrences  \
0  simon brodkin  [Q15072726] 2010-03-23 08:21:11               2   

                                              probas  \
0  [[simon brodkin, 0.4163], [isy suttie, 0.3932]...   

                                                urls phase country_speaker  \
0  [http://herefordtimes.com/leisure/theatre/5070...     A  United Kingdom   

   n_appearances country_urls  
0              1       [None]  
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
             quoteID                                          quotation  \
0  2010-10-15-113668  why testicular cancer can be cured with conven...   

         speaker       qids                date  numOccurrences  \
0  harold varmus  [Q295691] 2010-10-15 15:07

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
             quoteID                                          quotation  \
0  2011-08-11-012080  despite the name of the band and everything it...   

        speaker        qids                date  numOccurrences  \
0  dave schools  [Q5229775] 2011-08-11 23:39:00               1   

                                     probas  \
0  [[dave schools, 0.6847], [None, 0.3153]]   

                                             urls phase  \
0  [http://jambase.com/headsup.asp?storyID=51133]     C   

            country_speaker  n_appearances                country_urls  
0  United States of America              1  [United States of America]  
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
             quoteID                                          quotation  \
0  2011-01-12

             quoteID                                          quotation  \
0  2011-01-12-002045  advancements made with the cystic fibrosis res...   

         speaker         qids                date  numOccurrences  \
0  nancy clayton  [Q21484085] 2011-01-12 16:22:30               1   

                                      probas  \
0  [[nancy clayton, 0.5061], [None, 0.4939]]   

                                                urls phase  country_speaker  \
0  [http://niagarathisweek.com/community/communit...     C              NaN   

   n_appearances country_urls  
0              1       [None]  
             quoteID                                          quotation  \
0  2011-06-01-001709              absolute shock and dismay and sorrow,   
1  2011-11-16-001977  activision blizzard was much better than conse...   
2  2011-04-15-002218  after that, they can be expected to happen onc...   
3  2011-10-24-008401  canst thou quake and change thy color / murder...   
4  2011-03-16-0

...

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
             quoteID                                          quotation  \
0  2014-06-12-000806  A huge thank you to everyone who participated,...   
1  2014-02-23-003828  Both of these guys had three chances to be in ...   
2  2014-10-20-008024  Breast Cancer Awareness begins with self exami...   

              speaker       qids                date  numOccurrences  \
0                None         [] 2014-06-12 02:25:00               1   
1        Gene Simmons  [Q207197] 2014-02-23 21:56:00               1   
2  Olivia Newton-John  [Q185165] 2014-10-20 20:45:09               1   

                              

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
             quoteID                                          quotation  \
0  2014-04-02-000362  A connected world has unlocked opportunities b...   
1  2014-09-11-004138  An ISIS attack in the homeland is an inevitabi...   
2  2014-06-27-002267  al Qaeda is no longer seen as an existential t...   
3  2014-09-15-003418  And beginning in late 2007, early 2008, a very...   
4  2014-01-03-001991  all the more repugnant in the context of the S...   
5  2014-01-13-003679  And the NSA has also exaggerated the role of i...   
6  2014-09-09-006139  As of late, he's also raising the flag with re...   
7  2014-09-10-009809  But as the President was focused on unwinding ...   
8  2014-07-19-003873  Boko Haram attacks have become re-occurring in...   

           speaker                             qids                date  \
0          Eric Xu           [Q19841061, Q59219265] 2014-04-

             quoteID                                          quotation  \
0  2015-04-01-000473  a credit to the Nigerian people and a truly hi...   
1  2015-10-27-005186  Al Qaeda and the Al Qaeda affiliated organizat...   
2  2015-09-01-004264  All I have to do is find the Prescott grave, a...   
3  2015-12-01-004963  all the promises that oh yes, now we're going ...   
4  2015-03-26-030518  I have not talked to the (O's minor league) st...   
5  2015-01-15-006429  As this week's transfer demonstrates, the admi...   
6  2015-12-16-009725  at a time when the fiscal outlook looks challe...   
7  2015-11-17-010237  At the same time, we can't completely abandon ...   
8  2015-05-21-005792  Base flood elevation is the number one hauntin...   

               speaker                                               qids  \
0        David Cameron  [Q1173911, Q16195447, Q18719363, Q192, Q473455...   
1  Michel Chossudovsky                                          [Q731787]   
2         Richard 

             quoteID                                          quotation  \
0  2016-09-29-002809  A lot of us hold leadership positions and that...   

  speaker qids                date  numOccurrences  \
0    None   [] 2016-09-29 19:17:57               1   

                                    probas  \
0  [[None, 0.9153], [Joe Manchin, 0.0847]]   

                                                urls phase country_speaker  \
0  [http://wvva.com/story/33273330/2016/09/28/sen...     E              []   

   n_appearances                country_urls  
0              1  [United States of America]  
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
             quoteID                                          quotation  \
0  2016-07-30-010613  He said that, unbeknownst to us, between the a...   

  speaker qids                date  numOccurrences  \
0    None   [] 2016-07-30 11:25:00               1   

                                        probas  \
0  [[None,

4  United States of America              2                      [None, None]  
             quoteID                                          quotation  \
0  2017-02-27-044295  Josephine Skriver attends the 25th Annual Elto...   

  speaker qids                date  numOccurrences  \
0    None   [] 2017-02-27 14:54:52               2   

                                   probas  \
0  [[None, 0.5193], [Elton John, 0.4807]]   

                                                urls phase country_speaker  \
0  [http://www.msn.com/en-us/movies/oscars/2017-o...     E              []   

   n_appearances  country_urls  
0              2  [None, None]  
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
             quoteID                                          quotation  \
0  2017-08-04-000683  100 bridges can be collapse anytime and they n...   
1  2017-11-25-037765  My eyes have been waiting for to see Sukhdev. ...   

          speaker                            

In [ ]:
quotebank.quotes_occurrences_df

To illustrate the results, we plot the number of quotes found for each issue in 2016

In [ ]:
quotebank.quotes_percentage_df = percentage_of_total_count(quotebank.quotes_occurrences_df, quotebank.get_all_keyword_names())

In [ ]:
quotebank.quotes_percentage_df

In [ ]:
occurences = list()
x_labels = list()
x_number = list()
count = 0
for key in quotebank.keywords:
    with open(key.output_filenames[8], 'rU') as f:
        key.quotes = pd.read_json(key.output_filenames[8], lines=True)
    occurences.append(len(key.quotes))
    x_labels.append(key.name)
    x_number.append(count)
    count += 1

In [ ]:
plt.bar(x_number, occurences, log = True)
plt.xticks(x_number, x_labels)
plt.xticks(rotation=90)
plt.title("Number of occurences per topic in 2016")
plt.xlabel("Topics")
plt.ylabel("Number of occurences")
plt.show()

In [ ]:
def freq_per_month(df):
    freqs = np.zeros(12)
    for i in range(12):
        freqs[i] = np.sum(df["date"].dt.month == i+1)
    return freqs

In [ ]:
all_freq = list()
for keys in quotebank.keywords:
    if not keys.quotes.empty:
        all_freq.append(freq_per_month(keys.quotes))
    else:
        all_freq.append(np.zeros(12))
df = pd.DataFrame(all_freq)

plt.plot(np.sum(df))
plt.title("Number of topic quotes found throughout the year 2016")
plt.show()

# Population data preprocessing
This is population data from the UN, which we might use in combination with the Our World in Data datasets to get a more realistic understanding of the relative deaths each year.

In [ ]:
population_path = DATA_PATH + "annual_population_data.csv"
raw_population_df = pd.read_csv(population_path)
raw_population_df

Extracting world population in millions from 2008 to 2017

In [ ]:
population_df = raw_population_df[(raw_population_df["Location"] == "World") & (raw_population_df["Time"] > 2007) & (raw_population_df["Time"] < 2018)]
population_df = population_df[["Time", "PopTotal"]].reset_index(drop=True)
population_df["PopTotal"] = population_df["PopTotal"].apply(lambda x: x/1000)
population_df = population_df.rename(columns = {"Time": "Year", "PopTotal": "World population millions"})
population_df

# Our World in Data preprocessing

This part of the notebook is more exploratory. We wanted to preprocess and look at the two main datasets form Our World in Data to be sure that the project is possible to do. 

## Preprocess death data

### Deaths by cause

In [ ]:
death_by_cause_path = DATA_PATH + "annual_number_of_deaths_by_cause.csv"
deaths_by_cause = Deaths("deaths_by_cause", death_by_cause_path, rename_cols = {'Terrorism (deaths)': 'Terrorism', 'Self-harm': 'Suicide'}, drop_cols = ['Number of executions (Amnesty International)'])
deaths_by_cause.df

In [ ]:
deaths_by_cause.df.columns

In [ ]:
deaths_by_cause.percentage_df

### Deaths by category

In [ ]:
death_by_category_path = DATA_PATH + "categorized_annual_number_of_deaths.csv"
deaths_by_category = Deaths("deaths_by_category", death_by_category_path)
deaths_by_category.df

In [ ]:
deaths_by_category.percentage_df

## Plotting

### Deaths by cause

In [ ]:
deaths_by_cause.plot_lines("Percentage deaths", "Percentage of deaths by cause")

### Deaths by category

In [ ]:
deaths_by_category.plot_stacked_areas("Percentage deaths", "Percentage of deaths by category")

# RQ1

In [ ]:
def plot_RQ1():
    

In [ ]:
quotebank.quotes_percentage_df.columns

In [ ]:
plot_RQ1()

# RQ5

In [ ]:
def group_quotebank_cols():
    print(":)")

In [ ]:
def plot_RQ5():
    bar_labels = ["quotes", "deaths"]
    width = 0.35 
    for year in range(BEGIN_YEAR, END_YEAR + 1):
        prev_values = [0, 0]
        print("Year:",  year)
        fig, ax = plt.subplots()

        for column in deaths_by_cause.quant_columns:
            values = [quotebank.quotes_percentage_df.loc[quotebank.quotes_percentage_df["Year"] == year, column].item(), 
                      deaths_by_cause.percentage_df.loc[deaths_by_cause.percentage_df["Year"] == year, column].item()]

            ax.bar(bar_labels, values, width, bottom = prev_values, label=column)
            prev_values[0] += values[0]
            prev_values[1] += values[1]

        ax.set_ylabel('Percentage')
        ax.set_title('Percentages of quotes and deaths')
        ax.legend()

        plt.show()

In [ ]:
deaths_by_cause.quant_columns

In [ ]:
plot_RQ5()